# Baseline

Input ▶ Data Preprocessing ▶ Training ▶ Model Evaluation ▶ Load the model ▶ Preprocess the data ▶ Inference

## Input

**Dataset AbdomenCT-1K**<br>
[Ma et al., 2021](https://arxiv.org/abs/2010.14808#:~:text=AbdomenCT%2D1K%3A%20Is%20Abdominal%20Organ%20Segmentation%20A%20Solved%20Problem%3F,-Jun%20Ma%2C%20Yao&text=With%20the%20unprecedented%20developments%20in,variability%20on%20many%20benchmark%20datasets.) - [AbodmenCT-1K](https://github.com/JunMa11/AbdomenCT-1K)

**INPUT.shape = (N, C, H, W, D)**<br>
N is batch size, C is channel, H is height, W is width, and D is depth of the INPUT.shape<br>

e.g. example:
- (N, 1, 225, 225, 1)
- (N, 1, 225, 225, 10)
- (N, 1, 225, 225, 15)

---

## Data Preprocessing

**Data Preprocessing**<br>
- CODE - TBA
- [Kaggle - inference](https://www.kaggle.com/code/aritrag/kerascv-starter-notebook-infer)
- [MIDL2021-CT-Classification](https://github.com/vinbigdata-medical/MIDL2021-CT-Classification) - [Preprocess DICOM image](https://github.com/vinbigdata-medical/MIDL2021-CT-Classification#preprocess-dicom-image)

---

## Training

What is the task?
- Multilabel Multiclass Classification
- Ensemble (Binary, Multiclass) Classification
- Segmentation -> Classification

What is the dimension of the inputs?
- 2D, 2.5D, 3D

Which framework should we use?
- Keras
- PyTorch
- Tensorflow

Which data augmentation should we use?
- Augmentation

Which model should we use?
- ResNet, EfficientNet, MobileNet with pretrained weights



Model Review
---
Task / time / input / performance / reference / model why / limitation
--
|Model|Task|time|performance|reference|why|limitation|efficient framework|.   |
|------|---|---|---|---|---|---|---|---|
|EfficientNet 시리즈|classification|fast|모델 크기에 따라 다름|O|RNSA에서 많이 씀|데이터 셋 크기 주의|pytorch/tensorflow|EfficientNet-B6? EfficientNet-B5? EfficientNetV2?
|VGG 시리즈|classification/object detection|slow|대규모 효과적|O|multiclass 많이 씀|오래걸림|pytorch/tensorflow|
|ResNet 시리즈|classification/object detection/semantic segmentation|normal|good|O|RSNA에서 많이 씀|계층 주의|pytorch/tensorflow|ResNet101, ResNet152|
|Unet|segmentation|normal|good|O|의료분야 많이씀|데이터셋, 라벨|pytorch/tensorflow|다른 구조의 백본이랑 같이..
|GoogLeNet(Inception)|classification/object detection|fast|가볍다|O|의료분야 씀|복잡, 오버피팅 주의|tensorflow|
|Xception|classficiation/object detection|can be slow|good|O|multiclass에 사용됨|깊은 구조|tensorflow/keras
|DenseNet|classficiation/object detection|normal|good|O|RNSA/파라미터 공유|시간 주의|pytorch
|FusionNet|3D image object classification|마다 다름|multi modal|O|keras참고|데이터 표준화 주의|all

### 1. Kaggle

ML Model - [Abdominal Trauma Detection - ResNet-152 - Pytorch](https://www.kaggle.com/code/witoldnowogrski/abdominal-trauma-detection-resnet-152-pytorch)<br>
Tabular data -

### 2. Competition - RSNA

[Spine Fracture Detection](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/365115)

**1st**
- 3D segmentation : Resnet-18 + Unet +EffNetV2-Small backbone
- 2.5D classification : EffNetV2-Small, ConvNext Tiny, ConvNext Nano, NFNet-L0

**2nd** 2.5D, 3channel

- stage1： 2.5D CNN + Unet for Segmentation (efficientnet-b0 backbone)
- stage2： CNN + BiGRU + Attention mechanism for Classification

**3rd**
- Architecture : 2.5D CNN + 1D RNN
- backbone : resnest50d + seresnext50

**4th**
- 3D segmentation : Unet + ir-CSN-50
- 3D classification : ir(ip)-CSN-152

**5th**
- EfficientNet-B5 (Classification)
- EfficientNet-B3-UNet (Segmentation)

[Mammography Breast Cancer Detection](https://www.kaggle.com/competitions/rsna-breast-cancer-detection/discussion/392449)
- 4 x Convnextv1-small 2048x1024, validated on 4-folds splits
- YOLOX

[Covid19 Detection](https://www.kaggle.com/competitions/siim-covid19-detection/discussion/263658)

**1st** Multi-task classification + segmentation

- Ensemble of 4 models (Yolov5X6 input size 768 + EfficientDet D7 input size 768 + FasterRNN FPN resnet101 input size 1024 + FasterRNN FPN resnet200 input size 768) using weighted boxes fusion (IoU 0.6)

**3rd** hybrid classification-segmentation model

- Classification : architecture used EfficientNet-B6 and DeepLabV3+ while the other used Swin Transformer and FPN
- Segmentation : EfficientDet + MMDetection (ffDet-D7X, EffDet-D6, Swin-RepPoints)

[VinBigData Chest X-ray Abnormalities Detection](https://)

**1st** : Ensembling Detectron2 + YOLOv5

**2nd** : Ensembling YOLOv5, ResNet101, ResNet152, and HourGlass

### 3. We can adjust the backbone with other models

```python
def build_model(warmup_steps, decay_steps):
    # Define Input
    inputs = keras.Input(shape=config.IMAGE_SIZE + [3,], batch_size=config.BATCH_SIZE)
    
    # Define Backbone
    backbone = keras_cv.models.ResNetBackbone.from_preset("resnet50_imagenet")
    backbone.include_rescaling = False
    x = backbone(inputs)
    
    # GAP to get the activation maps
    gap = keras.layers.GlobalAveragePooling2D()
    x = gap(x)

    # Define 'necks' for each head
    x_bowel = keras.layers.Dense(32, activation='silu')(x)
    x_extra = keras.layers.Dense(32, activation='silu')(x)
    x_liver = keras.layers.Dense(32, activation='silu')(x)
    x_kidney = keras.layers.Dense(32, activation='silu')(x)
    x_spleen = keras.layers.Dense(32, activation='silu')(x)

    # Define heads
    out_bowel = keras.layers.Dense(1, name='bowel', activation='sigmoid')(x_bowel) # use sigmoid to convert predictions to [0-1]
    out_extra = keras.layers.Dense(1, name='extra', activation='sigmoid')(x_extra) # use sigmoid to convert predictions to [0-1]
    out_liver = keras.layers.Dense(3, name='liver', activation='softmax')(x_liver) # use softmax for the liver head
    out_kidney = keras.layers.Dense(3, name='kidney', activation='softmax')(x_kidney) # use softmax for the kidney head
    out_spleen = keras.layers.Dense(3, name='spleen', activation='softmax')(x_spleen) # use softmax for the spleen head
    
    # Concatenate the outputs
    outputs = [out_bowel, out_extra, out_liver, out_kidney, out_spleen]

    # Create model
    print("[INFO] Building the model...")
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Cosine Decay
    cosine_decay = keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1e-4,
        decay_steps=decay_steps,
        alpha=0.0,
        warmup_target=1e-3,
        warmup_steps=warmup_steps,
    )

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=cosine_decay)
    loss = {
        "bowel":keras.losses.BinaryCrossentropy(),
        "extra":keras.losses.BinaryCrossentropy(),
        "liver":keras.losses.CategoricalCrossentropy(),
        "kidney":keras.losses.CategoricalCrossentropy(),
        "spleen":keras.losses.CategoricalCrossentropy(),
    }
    metrics = {
        "bowel":["accuracy"],
        "extra":["accuracy"],
        "liver":["accuracy"],
        "kidney":["accuracy"],
        "spleen":["accuracy"],
    }
    print("[INFO] Compiling the model...")
    model.compile(
        optimizer=optimizer,
      loss=loss,
      metrics=metrics
    )
    
    return model
```

### 4. Literature Review

1. [MIDL2021-CT-Classification](https://github.com/vinbigdata-medical/MIDL2021-CT-Classification)
2. [GP LEE et al. 2023](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10241572/#:~:text=PMID%3A%2037284168-,Enhancing%20Disease%20Classification%20in%20Abdominal%20CT%20Scans%20through%20RGB%20Superposition,Study%20of%20Appendicitis%20and%20Diverticulitis) - [PDF](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10241572/pdf/CMMM2023-7714483.pdf)
3. [Liu et al., 2023](https://github.com/ljwztc/CLIP-Driven-Universal-Model) - [arXiv](https://arxiv.org/abs/2301.00785)
4. [Ma et al., 2021](https://arxiv.org/abs/2010.14808#:~:text=AbdomenCT%2D1K%3A%20Is%20Abdominal%20Organ%20Segmentation%20A%20Solved%20Problem%3F,-Jun%20Ma%2C%20Yao&text=With%20the%20unprecedented%20developments%20in,variability%20on%20many%20benchmark%20datasets.) - [AbodmenCT-1K](https://github.com/JunMa11/AbdomenCT-1K)

---

## Model Evaluation

How to compare the model performance?
- GroupKFold (K = 5) only for performance check
- patientID as group, elements are seriesID







---

## Load the model

Follow the training infer reference:<br>
- [Train](https://www.kaggle.com/code/aritrag/kerascv-starter-notebook-train)
- [Infer](https://www.kaggle.com/code/jihoonkim2100/kerascv-starter-notebook-infer)


---

## Preprocess the data

**Data Preprocessing**<br>
- CODE - TBA
- [Reference](https://www.kaggle.com/code/aritrag/kerascv-starter-notebook-infer)

---

## Inference

Inference

---

# Score